# Evolver Loop 2 Analysis

## Objectives:
1. Analyze current experiment results (exp_001: 117.28)
2. Check available snapshot solutions for ensemble potential
3. Understand the gap to target (68.93)
4. Identify the best path forward

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os
import glob

getcontext().prec = 25
scale_factor = Decimal("1e18")

print("Libraries loaded")

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Scoring functions
def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_side_length(trees):
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side**2 / n

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    scores_by_n = {}
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            score = get_score(trees, n)
            total_score += score
            scores_by_n[n] = score
    return total_score, scores_by_n

print("Scoring functions defined")

Scoring functions defined


In [4]:
# Find all available CSV files
csv_files = []

# Current experiment candidates
csv_files.extend(glob.glob('/home/code/submission_candidates/*.csv'))
csv_files.extend(glob.glob('/home/code/experiments/*/submission*.csv'))
csv_files.extend(glob.glob('/home/code/experiments/*/*.csv'))

# Snapshot files
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission*.csv'))
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission_candidates/*.csv'))
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/*.csv'))

# Sample submission
csv_files.append('/home/data/sample_submission.csv')

# Deduplicate
csv_files = list(set(csv_files))
print(f"Found {len(csv_files)} CSV files to analyze")

Found 58 CSV files to analyze


In [5]:
# Score all available submissions
results = []

for csv_file in csv_files:
    try:
        if os.path.exists(csv_file) and os.path.getsize(csv_file) > 10000:  # At least 10KB
            score, scores_by_n = score_submission(csv_file)
            results.append({
                'file': csv_file,
                'score': score,
                'scores_by_n': scores_by_n
            })
            print(f"{score:.4f} - {csv_file.split('/')[-1]}")
    except Exception as e:
        print(f"Error with {csv_file}: {e}")

results.sort(key=lambda x: x['score'])
print(f"\nBest score: {results[0]['score']:.6f} from {results[0]['file']}")
print(f"Worst score: {results[-1]['score']:.6f} from {results[-1]['file']}")

125.2911 - best_seed0.csv


118.1934 - submission_v21.csv


87.8040 - submission.csv


87.8968 - candidate_001.csv


135.8191 - submission_final.csv


...

217.5762 - candidate_000.csv


164.0895 - candidate_000.csv


122.0719 - submission_optimized.csv


122.0719 - submission.csv


149.2940 - candidate_000.csv

Best score: 87.364112 from /home/nonroot/snapshots/santa-2025/20992150197/code/submission_candidates/candidate_002.csv
Worst score: 217.576225 from /home/nonroot/snapshots/santa-2025/20992150197/code/submission_candidates/candidate_000.csv


In [6]:
# Analyze the best submissions
print("\n" + "="*60)
print("TOP 5 SUBMISSIONS")
print("="*60)
for i, r in enumerate(results[:5]):
    print(f"{i+1}. Score: {r['score']:.6f} - {r['file'].split('/')[-1]}")

print("\n" + "="*60)
print("SCORE BREAKDOWN")
print("="*60)
print(f"Target: 68.931058")
print(f"Best available: {results[0]['score']:.6f}")
print(f"Gap to target: {results[0]['score'] - 68.931058:.6f}")
print(f"Percentage worse: {(results[0]['score'] - 68.931058) / 68.931058 * 100:.1f}%")


TOP 5 SUBMISSIONS
1. Score: 87.364112 - candidate_002.csv
2. Score: 87.364112 - candidate_002.csv
3. Score: 87.364112 - candidate_002.csv
4. Score: 87.364112 - candidate_002.csv
5. Score: 87.364112 - candidate_002.csv

SCORE BREAKDOWN
Target: 68.931058
Best available: 87.364112
Gap to target: 18.433054
Percentage worse: 26.7%


In [7]:
# Build an ensemble from all available solutions
# For each N, pick the best configuration across all sources

print("\n" + "="*60)
print("ENSEMBLE ANALYSIS")
print("="*60)

# Load all dataframes
dfs = {}
for r in results:
    try:
        dfs[r['file']] = pd.read_csv(r['file'])
    except:
        pass

# For each N, find the best source
ensemble_scores = {}
ensemble_sources = {}

for n in range(1, 201):
    best_score = float('inf')
    best_source = None
    
    for file_path, df in dfs.items():
        trees = load_configuration_from_df(n, df)
        if trees and len(trees) == n:
            score = get_score(trees, n)
            if score < best_score:
                best_score = score
                best_source = file_path
    
    if best_source:
        ensemble_scores[n] = best_score
        ensemble_sources[n] = best_source

ensemble_total = sum(ensemble_scores.values())
print(f"Ensemble score (best per N): {ensemble_total:.6f}")
print(f"Improvement over best single: {results[0]['score'] - ensemble_total:.6f}")
print(f"Gap to target: {ensemble_total - 68.931058:.6f}")


ENSEMBLE ANALYSIS


Ensemble score (best per N): 85.649334
Improvement over best single: 1.714778
Gap to target: 16.718276


In [8]:
# Analyze which sources contribute most to the ensemble
from collections import Counter

source_counts = Counter(ensemble_sources.values())
print("\nSource contributions to ensemble:")
for source, count in source_counts.most_common():
    print(f"  {count:3d} configs from {source.split('/')[-1]}")


Source contributions to ensemble:
  180 configs from candidate_002.csv
   13 configs from submission.csv
    7 configs from candidate_001.csv


In [9]:
# Analyze the gap by N - which N values are hardest?
print("\n" + "="*60)
print("HARDEST N VALUES (highest score contribution)")
print("="*60)

best_scores_by_n = results[0]['scores_by_n']
sorted_by_score = sorted(best_scores_by_n.items(), key=lambda x: x[1], reverse=True)

print("Top 20 worst N values:")
for n, score in sorted_by_score[:20]:
    print(f"  N={n:3d}: score={score:.6f} (side={np.sqrt(score*n):.4f})")

print(f"\nSum of top 20 worst: {sum(s for _, s in sorted_by_score[:20]):.6f}")
print(f"Percentage of total: {sum(s for _, s in sorted_by_score[:20]) / results[0]['score'] * 100:.1f}%")


HARDEST N VALUES (highest score contribution)
Top 20 worst N values:
  N=  6: score=0.666667 (side=2.0000)
  N=  1: score=0.661250 (side=0.8132)
  N=  5: score=0.639443 (side=1.7881)
  N= 13: score=0.603077 (side=2.8000)
  N=  7: score=0.600357 (side=2.0500)
  N= 15: score=0.600000 (side=3.0000)
  N=  2: score=0.598746 (side=1.0943)
  N= 16: score=0.562500 (side=3.0000)
  N=  3: score=0.560667 (side=1.2969)
  N= 14: score=0.560000 (side=2.8000)
  N=  8: score=0.551250 (side=2.1000)
  N= 11: score=0.545682 (side=2.4500)
  N=  4: score=0.544079 (side=1.4752)
  N=  9: score=0.537778 (side=2.2000)
  N= 17: score=0.529412 (side=3.0000)
  N= 28: score=0.529375 (side=3.8500)
  N= 19: score=0.522237 (side=3.1500)
  N= 31: score=0.514930 (side=3.9954)
  N= 29: score=0.511121 (side=3.8500)
  N= 12: score=0.500208 (side=2.4500)

Sum of top 20 worst: 11.338778
Percentage of total: 13.0%


In [10]:
# Summary and recommendations
print("\n" + "="*60)
print("SUMMARY AND RECOMMENDATIONS")
print("="*60)

print(f"""
1. CURRENT STATUS:
   - Best experiment score: 117.28 (exp_001)
   - Best available score: {results[0]['score']:.4f}
   - Ensemble potential: {ensemble_total:.4f}
   - Target: 68.93
   - Gap: {ensemble_total - 68.931058:.2f} ({(ensemble_total - 68.931058) / 68.931058 * 100:.1f}% worse)

2. KEY OBSERVATIONS:
   - Ensemble approach shows {results[0]['score'] - ensemble_total:.4f} improvement potential
   - Top 20 worst N values contribute {sum(s for _, s in sorted_by_score[:20]) / results[0]['score'] * 100:.1f}% of score
   - Need to reduce score by ~{results[0]['score'] - 68.931058:.1f} points to reach target

3. CRITICAL GAPS:
   - No bbox3 binary available (top kernels use this)
   - No external solution datasets (bucket-of-chump, etc.)
   - Limited optimization time compared to top kernels (3+ hours)

4. RECOMMENDED NEXT STEPS:
   a) Implement ensemble approach - combine best configs per N
   b) Run longer optimization (more iterations, more seeds)
   c) Focus on worst N values (high-impact optimization)
   d) Try to obtain/compile bbox3 or equivalent optimizer
""")


SUMMARY AND RECOMMENDATIONS

1. CURRENT STATUS:
   - Best experiment score: 117.28 (exp_001)
   - Best available score: 87.3641
   - Ensemble potential: 85.6493
   - Target: 68.93
   - Gap: 16.72 (24.3% worse)

2. KEY OBSERVATIONS:
   - Ensemble approach shows 1.7148 improvement potential
   - Top 20 worst N values contribute 13.0% of score
   - Need to reduce score by ~18.4 points to reach target

3. CRITICAL GAPS:
   - No bbox3 binary available (top kernels use this)
   - No external solution datasets (bucket-of-chump, etc.)
   - Limited optimization time compared to top kernels (3+ hours)

4. RECOMMENDED NEXT STEPS:
   a) Implement ensemble approach - combine best configs per N
   b) Run longer optimization (more iterations, more seeds)
   c) Focus on worst N values (high-impact optimization)
   d) Try to obtain/compile bbox3 or equivalent optimizer



In [11]:
# Check the best snapshot solution in detail
best_snapshot = '/home/nonroot/snapshots/santa-2025/20992150197/code/submission_candidates/candidate_002.csv'
df_best = pd.read_csv(best_snapshot)
print(f"Best snapshot: {best_snapshot}")
print(f"Score: 87.364112")
print(f"Rows: {len(df_best)}")
print(f"\nFirst 10 rows:")
print(df_best.head(10))

Best snapshot: /home/nonroot/snapshots/santa-2025/20992150197/code/submission_candidates/candidate_002.csv
Score: 87.364112
Rows: 20100

First 10 rows:
      id           x           y          deg
0  001_0        s0.0        s0.0        s45.0
1  002_0    s-0.0255     s0.0493         s0.0
2  002_1     s0.3688     s0.7431       s180.0
3  003_0   s0.431989   s-0.16564   s63.434948
4  003_1   s0.745039   s0.460459   s63.434948
5  003_2  s-0.127028   s0.505181  s243.434948
6  004_0   s0.124855  s-0.124889   s15.376252
7  004_1   s0.799799    s0.06072   s15.376252
8  004_2   s0.250201    s0.73928  s195.376252
9  004_3   s0.925145   s0.924889  s195.376252
